In [1]:
#importing libaries
from keras.preprocessing import text
from tensorflow.python.keras.utils import np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [5]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2)) #to learn vector representation of words
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,))) #to cal mean of word embeddings
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.6755886077881

Epoch: 2 	Loss: 429.2639682292938

Epoch: 3 	Loss: 426.0253610610962

Epoch: 4 	Loss: 422.92547821998596

Epoch: 5 	Loss: 420.4486451148987



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).tail()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
surpassing,0.014637,-0.036032,-0.046749,-0.047863,-0.035758,-0.013988,0.023638,-0.033054,0.029870,-0.017284,...,0.017053,0.028999,-0.039708,-0.015831,0.012468,0.038576,0.012354,-0.029725,-0.021488,0.010779
human,0.021684,0.033096,-0.048419,-0.021959,-0.048840,-0.000609,0.023838,-0.020833,-0.013493,0.038367,...,-0.014265,0.018466,0.032755,-0.042253,-0.026483,0.016066,-0.019283,-0.043340,-0.048455,0.032958
expert,0.002818,0.048112,0.004535,-0.006850,-0.037657,0.008355,-0.022439,-0.043486,-0.014944,0.029260,...,0.012235,0.009482,-0.026425,-0.042670,-0.009893,0.016643,0.047683,0.013361,0.025285,0.017481
performance,0.008167,-0.028454,-0.008894,0.012220,-0.023772,0.039075,0.003177,0.046515,-0.004720,-0.038140,...,-0.019270,-0.044338,-0.006905,-0.017143,-0.010305,-0.027216,0.047273,-0.034787,-0.026864,-0.006642
PAD,0.033471,-0.036667,0.043702,-0.010844,-0.048454,0.038698,-0.002277,-0.005137,-0.009646,0.038922,...,-0.032644,-0.019357,0.041913,0.024659,-0.011921,-0.016123,0.031209,-0.026904,0.010218,-0.014065


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['structured', 'on', 'translation', 'produced', 'they']}